In [ ]:
import os, sys
sys.path.append(os.path.abspath(os.environ['KLSCRIPTS']))
from klscripts.model.jupyter import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

Tools.git_commit_hash(os.environ['KLSCRIPTS'])

In [ ]:
import numpy as np
# conda install opencv
# If this isn't installed, will silently never open a video:
# opencv-contrib-python
# pip install --default-timeout=300 opencv-contrib-python
import cv2


def m(threshold):
    diffs = frame - prev > threshold
    return np.count_nonzero(diffs)

class StreamFeature:
    def __init__(self, path: str):
        self._path = path
        self._cap = cv2.VideoCapture(path)
    
    def stream(self, length: int, roi):
        if not self._cap.isOpened(): raise ValueError('TODO')
        _, raw = self._cap.read()
        prev = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY)
        vals = self.init_vals(length)
        i = 0
        while(self._cap.isOpened()):
            _, raw = self._cap.read()
            frame = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY)
            val = self.calc(self.crop(frame, roi), self.crop(prev, roi))
            vals = self.combine_vals(vals, val, i)
            prev = frame
            i += 1
            if i > 10000: return vals
        return vals
    
    def crop(self, frame, roi):
        r0,r1,c0,c1 = roi
        #print(roi, frame.shape)
        #raise ValueError()
        return frame[r0:r1,c0:c1]
    
    def init_vals(self, length):
        raise NotImplementedError()
        
    def combine_vals(self, prev_vals, next_val, i):
        raise NotImplementedError()
    
    def calc(self, frame, prev):
        raise NotImplementedError()
    
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self._cap.release()


class MiFeature(StreamFeature):
    def init_vals(self, length):
        return np.zeros(length - 1)
    def combine_vals(self, prev_vals, next_val, i):
        prev_vals[i] = next_val
    def calc(self, frame, prev):
        pass

    
class MiHistFeature(StreamFeature):
    
    def init_vals(self, length):
        return np.zeros(200+256)
    
    def combine_vals(self, prev_vals, next_val, i):
        prev_vals += next_val
        return prev_vals
        
    def calc(self, frame, prev):
        return np.bincount((200 + frame.astype(np.int16) - prev.astype(np.int16)).flatten(), minlength=200+256)


In [ ]:

path = '/Users/student/valar-cache/videos/2255/x265-crf15/x265-crf15.mkv'
hist_top = MiHistFeature(path).stream(20500, (0, int(round(796/2)), 0, 1180))
hist_bottom = MiHistFeature(path).stream(20500, (int(round(796/2)), 796, 0, 1180))

In [ ]:
left = 200 - 60
right = 200 + 60
fig = plt.figure(figsize=(30, 30))
ax = fig.add_subplot(1, 1, 1)
sns.barplot(np.arange(left, right), hist_top[left:right], ax=ax, color='#ff0000', alpha=.4)
sns.barplot(np.arange(left, right), hist_bottom[left:right], ax=ax, color='#0000ff', alpha=.4)
ax.set_ybound(0, 5000000)  # 500000000
fig
pass

In [ ]:
center = 200

left = center - 32
right = center + 32
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
sns.barplot(np.arange(left, right), np.log10(hist_bottom / hist_top)[left:right], ax=ax, color='#000000')
fig
pass

